# Preprocesamiento

## Extracción de texto

**pdfplumber**

https://pypi.org/project/pdfplumber/


Works best on machine-generated, rather than scanned, PDFs.

In [1]:
import pdfplumber

path = r"C:\Users\eliag\OneDrive - Universidad Politécnica de Madrid\CUATRI7\TFG\PREPROCESSING\basketball_notes.pdf"

In [2]:
 # Nombre del archivo de salida
archivo_salida = "salida.txt"
 
# Abre el archivo de salida en modo escritura con codificación UTF-8
with open(archivo_salida, "w", encoding="utf-8") as archivo:
    # Abre el archivo PDF
    with pdfplumber.open(path) as pdf:
        # Itera sobre las páginas del PDF y escribe directamente en el archivo
        for num_pagina, pagina in enumerate(pdf.pages, start=1):
            texto_pagina = pagina.extract_text()
            if texto_pagina:
                archivo.write(f"--- Página {num_pagina} ---\n")
                archivo.write(texto_pagina + "\n\n")  # Añade salto de línea entre páginas
 
print(f"El texto ha sido guardado en '{archivo_salida}'.")

El texto ha sido guardado en 'salida.txt'.


In [3]:
archivo_salida

'salida.txt'

## Resolución de correferencias

In [7]:
import spacy
from tqdm import tqdm

class CoreferencesResolver:
    def __init__(self, input_file, output_file):
        self.input_file = input_file
        self.output_file = output_file
        self.nlp = spacy.load("en_core_web_trf")  

    def replace_references(self, doc):
        """ Reemplaza menciones por su referencia principal. """
        token_mention_mapper = {}
        resolved_text = ""

        # Obtener los clusters de correferencia
        clusters = [val for key, val in doc.spans.items() if key.startswith("coref_cluster")]

        for cluster in clusters:
            main_mention = cluster[0]  # Referencia principal
            main_text = main_mention.text

            for mention in cluster[1:]:  # Omitir la principal
                token_mention_mapper[mention[0].idx] = main_text + mention[0].whitespace_
                for token in mention[1:]:
                    token_mention_mapper[token.idx] = ""  # Eliminar menciones secundarias

        # Construir el texto resuelto
        for token in doc:
            resolved_text += token_mention_mapper.get(token.idx, token.text + token.whitespace_)

        return resolved_text

    def process_text(self):
        """ Procesa el texto: resuelve correferencias y segmenta en oraciones. """
        with open(self.input_file, "r", encoding="utf-8") as f:
            text = f.read()

        doc = self.nlp(text)
        resolved_text = self.replace_references(doc)

        # Volver a procesar el texto corregido
        doc = self.nlp(resolved_text)
        sentences = [sent.text for sent in doc.sents]  # Segmentación en oraciones

        # Guardar en archivo
        with open(self.output_file, "w", encoding="utf-8") as f:
            for sentence in sentences:
                f.write(sentence + "\n")

        print(f"✅ Preprocesamiento completado. Guardado en '{self.output_file}'.")

# Ejecutar el preprocesamiento
if __name__ == "__main__":
    input_file = "salida.txt"
    output_file = "preprocesado.txt"

    resolver = CoreferencesResolver(input_file, output_file)
    resolver.process_text()


✅ Preprocesamiento completado. Guardado en 'preprocesado.txt'.
